# DenseNet Model Building Pipeline for 1D Signals with DEMO
DenseNet121, DenseNet161, DenseNet169, DenseNet201, DenseNet264

# Test GPU (Optional)
Before Starting, kindly check the available GPU from the Google Server, GPU model and other related information. It might help!

In [2]:
import torch
print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
print("GPU Number:", torch.cuda.device_count())
print("Current GPU Index:", torch.cuda.current_device())
print("GPU Type:", torch.cuda.get_device_name(device=None))
print("GPU Capability:", torch.cuda.get_device_capability(device=None))
print("Is GPU Initialized yet?", torch.cuda.is_initialized())

Is CUDA enabled GPU Available? True
GPU Number: 1
Current GPU Index: 0
GPU Type: Tesla T4
GPU Capability: (7, 5)
Is GPU Initialized yet? True


# Connect to Google Drive (Optional for Google COLAB)

Copy-Paste the Authorization Code and Mount Google Drive to COLAB

In [3]:
from google.colab import drive
drive.mount('/content/GDrive')

Mounted at /content/GDrive


Move to the Target Directory

In [5]:
%cd /content

/content/GDrive/MyDrive/Colab_Notebooks/GitHub/GitHub Files


List the Files and Folders Located in the Current Directory

In [6]:
!ls

DenseNet_1DCNN_Implementation_DEMO.ipynb   MobileNet_1DCNN.py
DenseNet_1DCNN.py			   ResNet_1DCNN.py
Drug_Persistency.gsheet			   ResNeXt_1DCNN.py
EfficientNet_1DCNN.py			   trained_models
Inception_1DCNN_Implementation_DEMO.ipynb  VGG_1DCNN.py
Inception_1DCNN.py			   WeatherHistory.csv
Inception_ResNet_1DCNN.py


Upload Files from Local Directory (if required)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving WeatherHistory.csv to WeatherHistory.csv


#Import Necessary Libraries

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import interp
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score, roc_curve, plot_roc_curve
from sklearn.metrics import confusion_matrix, classification_report, auc, precision_recall_curve, average_precision_score
sns.set_theme(style="whitegrid")

In [8]:
# Import DenseNet Module
from DenseNet_1DCNN import DenseNet

# DEMO: Regression and Classification


## Regression

### Import and Prepare Dataset

Import Dataset from a CSV file using Pandas. This DEMO uses the Weather History dataset from Kaggle.  
Source: https://www.kaggle.com/budincsevity/szeged-weather

In [40]:
dataset = pd.read_csv('WeatherHistory.csv')
print(dataset.shape)
dataset.head(5)

(96453, 11)


,Formatted Date,Summary,Precip Type,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Daily Summary,Temperature (C),Apparent Temperature (C)
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,0.89,14.1197,251,15.8263,1015.13,Partly cloudy throughout the day.,9.472222,7.388889
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,0.86,14.2646,259,15.8263,1015.63,Partly cloudy throughout the day.,9.355556,7.227778
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,0.89,3.9284,204,14.9569,1015.94,Partly cloudy throughout the day.,9.377778,9.377778
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,0.83,14.1036,269,15.8263,1016.41,Partly cloudy throughout the day.,8.288889,5.944444
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,0.83,11.0446,259,15.8263,1016.51,Partly cloudy throughout the day.,8.755556,6.977778


Have a look at the dataset! The CSV file includes a hourly/daily summary for Szeged, Hungary area, between 2006 and 2016.

Data available in the hourly response:

1. Time  
2. Summary  
3. PrecipType  
4. Temperature  
5. ApparentTemperature  
6. Humidity  
7. WindSpeed  
8. WindBearing  
9. Visibility  
10. LoudCover  
11. Pressure  

Check for any existing 'NaN' value in the dataset which might create issues afterwards. We, here, check both overall and column by column.

In [41]:
print(f'Total NULL values in the Original DataFrame = {dataset.isnull().sum().sum()}')
print(dataset.isnull().sum())

Total NULL values in the Original DataFrame = 517
Formatted Date                0
Summary                       0
Precip Type                 517
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Pressure (millibars)          0
Daily Summary                 0
Temperature (C)               0
Apparent Temperature (C)      0
dtype: int64


There are total 517 NULL or NaN values and all of them are in the 'Precip_Type' Column. Let's print out the unique values in that column.

In [42]:
dataset['Precip Type'].unique()

array(['rain', 'snow', nan], dtype=object)

So, it can be seen that there are mainly 2 classes in this column other than the NaN. So, we can replace the NaN cell with 'Unknown' instead of '0' (which is done more commonly).

In [43]:
dataset['Precip Type'] = dataset['Precip Type'].fillna('Unknown')

One important realization is that we could easily remove those rows in this case instead of filling up the NULLs, but that would remove some datapoints. For very large datasets, it might not matter. On the contrary, sometimes it might remove important observations. If the dataset is too sparse, then it is better to remove the rows instead of filling. But since in this case, value in only 1 column was missing, it was filled up with a suitable replacement.

In [44]:
dataset['Precip Type'].unique()

array(['rain', 'snow', 'Unknown'], dtype=object)

Now, at first slice the DataTime Frame into month and hour since they can be strong indicators of weather (fairly long term and short term). Years can also show gradual change in weather pattern (especially change in the climate) but that has been ignored in this study. Modify it according to your wish to improve performance.

In [45]:
dataset['Month'] = (dataset['Formatted Date']).str.slice(start=5, stop=7)
dataset['Hour'] = (dataset['Formatted Date']).str.slice(start=11, stop=13)
dataset.head(5)

,Formatted Date,Summary,Precip Type,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Daily Summary,Temperature (C),Apparent Temperature (C),Month,Hour
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,0.89,14.1197,251,15.8263,1015.13,Partly cloudy throughout the day.,9.472222,7.388889,04,00
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,0.86,14.2646,259,15.8263,1015.63,Partly cloudy throughout the day.,9.355556,7.227778,04,01
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,0.89,3.9284,204,14.9569,1015.94,Partly cloudy throughout the day.,9.377778,9.377778,04,02
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,0.83,14.1036,269,15.8263,1016.41,Partly cloudy throughout the day.,8.288889,5.944444,04,03
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,0.83,11.0446,259,15.8263,1016.51,Partly cloudy throughout the day.,8.755556,6.977778,04,04


The first DateTime Frame column and the column containing the real temperature for that hour have been dropped. The aim here is to predict "Apparent Temperature" from other weather indicators using Deep Learning based Regression.

In [46]:
dataset.drop(['Formatted Date', 'Temperature (C)'], axis=1, inplace=True)
dataset.head(5)

,Summary,Precip Type,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Daily Summary,Apparent Temperature (C),Month,Hour
0,Partly Cloudy,rain,0.89,14.1197,251,15.8263,1015.13,Partly cloudy throughout the day.,7.388889,04,00
1,Partly Cloudy,rain,0.86,14.2646,259,15.8263,1015.63,Partly cloudy throughout the day.,7.227778,04,01
2,Mostly Cloudy,rain,0.89,3.9284,204,14.9569,1015.94,Partly cloudy throughout the day.,9.377778,04,02
3,Partly Cloudy,rain,0.83,14.1036,269,15.8263,1016.41,Partly cloudy throughout the day.,5.944444,04,03
4,Mostly Cloudy,rain,0.83,11.0446,259,15.8263,1016.51,Partly cloudy throughout the day.,6.977778,04,04


Rearrange the Columns, place the target column at the end for next step

In [47]:
dataset = dataset[['Month', 'Hour', 'Summary', 'Precip Type', 'Humidity', 
                   'Wind Speed (km/h)', 'Wind Bearing (degrees)', 
                   'Visibility (km)', 'Pressure (millibars)', 'Daily Summary', 
                   'Apparent Temperature (C)']]
dataset.head(5)

,Month,Hour,Summary,Precip Type,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Daily Summary,Apparent Temperature (C)
0,04,00,Partly Cloudy,rain,0.89,14.1197,251,15.8263,1015.13,Partly cloudy throughout the day.,7.388889
1,04,01,Partly Cloudy,rain,0.86,14.2646,259,15.8263,1015.63,Partly cloudy throughout the day.,7.227778
2,04,02,Mostly Cloudy,rain,0.89,3.9284,204,14.9569,1015.94,Partly cloudy throughout the day.,9.377778
3,04,03,Partly Cloudy,rain,0.83,14.1036,269,15.8263,1016.41,Partly cloudy throughout the day.,5.944444
4,04,04,Mostly Cloudy,rain,0.83,11.0446,259,15.8263,1016.51,Partly cloudy throughout the day.,6.977778


At this stage, check the datatypes of the columns. You can see that 'Month' and 'Hour' columns are object types, convert them to numeric 'int64' (otherwise while creating dummy variables, each month or hour will get a separate column, which is unncessary). 

In [48]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     96453 non-null  object 
 1   Hour                      96453 non-null  object 
 2   Summary                   96453 non-null  object 
 3   Precip Type               96453 non-null  object 
 4   Humidity                  96453 non-null  float64
 5   Wind Speed (km/h)         96453 non-null  float64
 6   Wind Bearing (degrees)    96453 non-null  int64  
 7   Visibility (km)           96453 non-null  float64
 8   Pressure (millibars)      96453 non-null  float64
 9   Daily Summary             96453 non-null  object 
 10  Apparent Temperature (C)  96453 non-null  float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.1+ MB


In [49]:
dataset['Month'] = dataset['Month'].astype('int64')
dataset['Hour'] = dataset['Hour'].astype('int64')

In [50]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     96453 non-null  int64  
 1   Hour                      96453 non-null  int64  
 2   Summary                   96453 non-null  object 
 3   Precip Type               96453 non-null  object 
 4   Humidity                  96453 non-null  float64
 5   Wind Speed (km/h)         96453 non-null  float64
 6   Wind Bearing (degrees)    96453 non-null  int64  
 7   Visibility (km)           96453 non-null  float64
 8   Pressure (millibars)      96453 non-null  float64
 9   Daily Summary             96453 non-null  object 
 10  Apparent Temperature (C)  96453 non-null  float64
dtypes: float64(5), int64(3), object(3)
memory usage: 8.1+ MB


Convert Text Data into Dummy Variables for Machine Learning.  
It is important since Machine Learning models will not accept non-numerical data directly during training or testing.

In [51]:
dummy_dataset = pd.DataFrame()

In [52]:
for i in range(0,len(dataset.columns)):
  X = dataset[dataset.columns[i]]
  if type(X[0]) == str:
    Y = pd.get_dummies(X)
    dummy_dataset = pd.concat([dummy_dataset, Y], axis=1)
  else:
    dummy_dataset = pd.concat([dummy_dataset, X], axis=1)
#
dummy_dataset.shape

(96453, 252)

In [53]:
dummy_dataset.head(5)

,Month,Hour,Breezy,Breezy and Dry,Breezy and Foggy,Breezy and Mostly Cloudy,Breezy and Overcast,Breezy and Partly Cloudy,Clear,Dangerously Windy and Partly Cloudy,Drizzle,Dry,Dry and Mostly Cloudy,Dry and Partly Cloudy,Foggy,Humid and Mostly Cloudy,Humid and Overcast,Humid and Partly Cloudy,Light Rain,Mostly Cloudy,Overcast,Partly Cloudy,Rain,Windy,Windy and Dry,Windy and Foggy,Windy and Mostly Cloudy,Windy and Overcast,Windy and Partly Cloudy,Unknown,rain,snow,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Breezy and foggy starting in the evening.,Breezy and foggy until morning.,Breezy and mostly cloudy overnight.,...,Partly cloudy starting overnight continuing until afternoon.,Partly cloudy starting overnight continuing until evening and breezy in the evening.,Partly cloudy starting overnight continuing until evening and breezy in the morning.,Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,Partly cloudy starting overnight continuing until evening.,Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until morning.,Partly cloudy starting overnight continuing until night and breezy in the afternoon.,Partly cloudy starting overnight continuing until night and breezy in the morning.,Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until night.,Partly cloudy starting overnight.,Partly cloudy throughout the day and breezy in the afternoon.,Partly cloudy throughout the day and breezy in the evening.,Partly cloudy throughout the day and breezy in the morning.,Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,Partly cloudy throughout the day and breezy starting in the morning continuing until night.,Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,Partly cloudy throughout the day.,Partly cloudy until afternoon.,Partly cloudy until evening and breezy in the afternoon.,Partly cloudy until evening and breezy in the morning.,Partly cloudy until evening and breezy overnight.,Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,Partly cloudy until evening and breezy starting in the morning continuing until evening.,Partly cloudy until evening.,Partly cloudy until morning.,Partly cloudy until night and breezy in the afternoon.,Partly cloudy until night and breezy in the morning.,Partly cloudy until night and breezy starting in the morning continuing until afternoon.,Partly cloudy until night.,Rain throughout the day.,Rain until afternoon.,Rain until morning.,Windy in the afternoon.,Apparent Temperature (C)
0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.89,14.1197,251,15.8263,1015.13,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.388889
1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.86,14.2646,259,15.8263,1015.63,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.227778
2,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.89,3.9284,204,14.9569,1015.94,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9.377778
3,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.83,14.1036,26

At this stage, you can plot the dataset prior to check any pattern in the dataset or just for illustrative purposes.

Convert Pandas DataFrame into NumPy Arrays

In [54]:
X_Data = dummy_dataset.iloc[:,0:-1].values # All columns except the last are the predicting variables
Y_Data = dummy_dataset.iloc[:,-1].values # Last column (Weight Column in this case) is the label
print(X_Data.shape)
print(Y_Data.shape)

(96453, 251)
(96453,)


Train-Test Split

In [55]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X_Data, Y_Data, test_size=0.20, random_state=42)

The third axis is to show the number of channels, which is 1 in this case. If same labels were appropriate for more than 1 dataset, there would be more than 1 channel.

In [56]:
X_Train = np.expand_dims(X_Train, axis=2)
X_Test = np.expand_dims(X_Test, axis=2)

In [57]:
print(X_Train.shape, X_Test.shape)
print(Y_Train.shape, Y_Test.shape)

(77162, 251, 1) (19291, 251, 1)
(77162,) (19291,)


### Build and Train Imported Data using the Regression Model

Configurations - Select the Configurations for the Regression Model. Vary them (while following the guidelines) to improve performance.

In [58]:
"Configurations for DenseNet in Regression Mode"
length = X_Train.shape[1]   # Number of Features (or length of the signal)
model_width = 32            # Number of Filter or Kernel in the Input Layer (Power of 2 to avoid error)
num_channel = 1             # Number of Input Channels
problem_type = 'Regression' # Regression or Classification
output_number = 1           # Number of Outputs in the Regression Mode - 1 input is mapped to a single output

Build Model

In [59]:
Regression_Model = DenseNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).DenseNet169() # Build Model
Regression_Model.compile(loss='mae', optimizer='adam', metrics= ['mse']) # Compile Model

Model_Summary

In [60]:
Regression_Model.summary() # Summary of the Model

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 251, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_168 (Conv1D)             (None, 126, 32)      256         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_168 (BatchN (None, 126, 32)      128         conv1d_168[0][0]                 
__________________________________________________________________________________________________
activation_168 (Activation)     (None, 126, 32)      0           batch_normalization_168[0][0]    
____________________________________________________________________________________________

Upload Past Weights if available

In [ ]:
Regression_Model.load_weights('Saved_Model.h5') # Load Previously Trained Weights for Transfer Learning

Train Model for 'n' number of Epochs with Batch size of 'm'

In [ ]:
# Early Stopping and Model_Checkpoints are optional parameters
# Early Stopping is to stop the training based on certain condition set by the user
# Model Checkpoint is to save a model in a directory based on certain conditions so that it can be used later for Transfer Learning or avoiding retraining
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, mode='min'), 
             tf.keras.callbacks.ModelCheckpoint('trained_models/DenseNet169_'+str(model_width)+'_'+str(num_channel)+'_'+str(output_number)+'_'+str(problem_type)+'.h5', 
                                                verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
history = Regression_Model.fit(X_Train, Y_Train, epochs=500, batch_size=128, verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks)
# Save 'History' of the model for model performance analysis performed later

Test and Predict

In [ ]:
# Preictions from the Test Set from the Trained Model
Predictions = Regression_Model.predict(X_Test, verbose=1)
print(Predictions.shape)

Error Performance (Mean Sqaured Error or MAE)

In [ ]:
# Error of the prediction, one of many evaluation metrics
# Using Mean Absolute Error (MAE) in this case as a sample
Error = mean_absolute_error(Y_Test, Predictions)
print(f"MAE: {Error}")

Plot Train and Validation Error and Loss

In [ ]:
def history_plot(history):
  # list all dictionaries in history
  print(history.history.keys())
  # summarize history for error
  plt.figure(figsize=(12,10))
  plt.subplot(2,1,1)
  plt.plot(history.history['mse'])
  plt.plot(history.history['val_mse'])
  plt.title('Model Error Performance')
  plt.ylabel('Error')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()
  # summarize history for loss
  plt.figure(figsize=(12,10))
  plt.subplot(2,1,2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()
#
history_plot(history)

Plot Prediction Performance

In [ ]:
def plot_prediction(test_labels, test_predictions):
  plt.figure(figsize=(15,10))
  plt.scatter(test_labels, test_predictions)
  '''Add Trendline'''
  z = np.polyfit(test_labels.ravel(), test_predictions.ravel(), 1)
  p = np.poly1d(z)
  plt.plot(test_labels, p(test_labels))
  plt.text(np.max(test_labels)/3,np.max(test_predictions),f'y = {p[1]:.2f}x+{p[0]:.2f}', fontsize=15)
  plt.title('Ground Truth vs. Prediction Scatter Plot', fontsize=20)
  plt.xlabel('Ground Truth', fontsize=15)
  plt.ylabel('Predictions', fontsize=15)
#
plot_prediction(Y_Test, Predictions)

Kernel Density Plot

In [ ]:
df = pd.DataFrame(data = {'Ground Truth': Y_Test.ravel(), 'Predictions': Predictions.ravel()})
plt.figure(figsize=(15,10))
sns.set_style('whitegrid')
sns.kdeplot(data=df)
plt.title('Kernel Density Estimation (KDE) Plot for Ground Truth and Predictions', fontsize=20)
plt.xlabel('Magnitude', fontsize=15)
plt.ylabel('Density', fontsize=15)

In [ ]:
plt.figure(figsize = (15,10))
df = pd.DataFrame(data = {'Ground Truth': Y_Test.ravel(), 'Predictions': Predictions.ravel()})
ax = sns.violinplot(data=df)
plt.title('Violin Plot for Ground Truth and Predictions', fontsize=20)
plt.ylabel('Magnitude', fontsize=15)

Both from the Kernel Density Plot or the Violin Plot, it can be seen that not only the MAE is good, but also the predicted values follow the same pattern as the ground truth, which proves the robustness of the model.

## Classification

### Data Preparation

Import the dataset from a CSV file and save it into a Pandas DataFrame

In [ ]:
dataset = pd.read_excel('Drug_Persistency.xlsx', sheet_name = 'Dataset')
dataset.drop(columns=['Ptid'], inplace=True) # Drop Patient ID Column
print(dataset.shape)
dataset.head(10)

Create a Blank DataFrame

In [28]:
dummy_dataset = pd.DataFrame()

In [29]:
for i in range(0,len(dataset.columns)):
  X = dataset[dataset.columns[i]]
  if type(X[0]) == str:
    Y = pd.get_dummies(X)
    dummy_dataset = pd.concat([dummy_dataset, Y], axis=1)
  else:
    dummy_dataset = pd.concat([dummy_dataset, X], axis=1)
#
dummy_dataset.head(10)

,Month,Hour,Breezy,Breezy and Dry,Breezy and Foggy,Breezy and Mostly Cloudy,Breezy and Overcast,Breezy and Partly Cloudy,Clear,Dangerously Windy and Partly Cloudy,Drizzle,Dry,Dry and Mostly Cloudy,Dry and Partly Cloudy,Foggy,Humid and Mostly Cloudy,Humid and Overcast,Humid and Partly Cloudy,Light Rain,Mostly Cloudy,Overcast,Partly Cloudy,Rain,Windy,Windy and Dry,Windy and Foggy,Windy and Mostly Cloudy,Windy and Overcast,Windy and Partly Cloudy,Unknown,rain,snow,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Breezy and foggy starting in the evening.,Breezy and foggy until morning.,Breezy and mostly cloudy overnight.,...,Partly cloudy starting overnight continuing until afternoon.,Partly cloudy starting overnight continuing until evening and breezy in the evening.,Partly cloudy starting overnight continuing until evening and breezy in the morning.,Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,Partly cloudy starting overnight continuing until evening.,Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until morning.,Partly cloudy starting overnight continuing until night and breezy in the afternoon.,Partly cloudy starting overnight continuing until night and breezy in the morning.,Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until night.,Partly cloudy starting overnight.,Partly cloudy throughout the day and breezy in the afternoon.,Partly cloudy throughout the day and breezy in the evening.,Partly cloudy throughout the day and breezy in the morning.,Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,Partly cloudy throughout the day and breezy starting in the morning continuing until night.,Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,Partly cloudy throughout the day.,Partly cloudy until afternoon.,Partly cloudy until evening and breezy in the afternoon.,Partly cloudy until evening and breezy in the morning.,Partly cloudy until evening and breezy overnight.,Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,Partly cloudy until evening and breezy starting in the morning continuing until evening.,Partly cloudy until evening.,Partly cloudy until morning.,Partly cloudy until night and breezy in the afternoon.,Partly cloudy until night and breezy in the morning.,Partly cloudy until night and breezy starting in the morning continuing until afternoon.,Partly cloudy until night.,Rain throughout the day.,Rain until afternoon.,Rain until morning.,Windy in the afternoon.,Apparent Temperature (C)
0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.89,14.1197,251,15.8263,1015.13,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.388889
1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.86,14.2646,259,15.8263,1015.63,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.227778
2,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.89,3.9284,204,14.9569,1015.94,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9.377778
3,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.83,14.1036,26

Convert the DataFrame to a Numpy Array

In [30]:
X_Data = dummy_dataset.iloc[:,0:-1].values
Y_Data = dummy_dataset.iloc[:,-1].values
print(X_Data.shape)
print(Y_Data.shape)

(96453, 251)
(96453,)


Get Unique Labels

In [31]:
'''Print out the Classes to track the sequence, will be important in the evaluation section (e.g., Labelling Conf Mat)'''
L_E = LabelEncoder()
L_E.fit_transform(Y_Data)
labels = L_E.classes_
print(f'Classes: {labels}')

Classes: [-27.71666667 -26.22777778 -25.92222222 ...  38.41666667  38.66111111
  39.34444444]


Train_Test Split

In [32]:
X_Train, X_Test, Y_Train_, Y_Test_ = train_test_split(X_Data, Y_Data, test_size=0.20, random_state=42)

One-Hot-Encoding for the Classification Labels

In [33]:
def one_hot_encoding(data):
  L_E = LabelEncoder()
  integer_encoded = L_E.fit_transform(data)  
  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  one_hot_encoded_data = onehot_encoder.fit_transform(integer_encoded)
  return one_hot_encoded_data

In [34]:
Y_Train = one_hot_encoding(Y_Train_.ravel())
Y_Test = one_hot_encoding(Y_Test_.ravel())

Train and Test Data Shapes

In [35]:
print(X_Train.shape, X_Test.shape)
print(Y_Train.shape, Y_Test.shape)

(77162, 251) (19291, 251)
(77162, 8721) (19291, 6405)


### Build and Train Imported Data using the Classification Model

Configurations

In [36]:
"Configurations for the 1D Network in Classification Mode"
length = X_Train.shape[1]       # Number of Features (or length of the signal)
model_width = 32                # Number of Filter or Kernels in the Input Layer (Power of 2 to avoid error)
num_channel = 1                 # Number of Input Channels
problem_type = 'Classification' # Regression or Classification
class_number = Y_Train.shape[1] # Number of Output Class in Classification Mode (>=2)

Build Model

In [37]:
Classification_Model = DenseNet(length, num_channel, model_width, problem_type=problem_type, output_nums=class_number).DenseNet169()
if class_number == 2:
  Classification_Model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mse','accuracy'])
elif class_number > 2:
  Classification_Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mse','accuracy'])

Model Summary

In [38]:
Classification_Model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 251, 1)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 126, 32)      256         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 126, 32)      128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 126, 32)      0           batch_normalization[0][0]        
______________________________________________________________________________________________

Upload Past Weights (Transfer Learning)

In [ ]:
Classification_Model.load_weights('Saved_Model.h5') # Load Previously Trained Weights for Transfer Learning

Train Model

In [ ]:
# Early Stopping and Model_Checkpoints are optional parameters
# Early Stopping is to stop the training based on certain condition set by the user
# Model Checkpoint is to save a model in a directory based on certain conditions so that it can be used later for Transfer Learning or avoiding retraining
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, mode='min'), 
             tf.keras.callbacks.ModelCheckpoint('Saved_Model.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
history = Classification_Model.fit(X_Train, Y_Train, epochs=300, batch_size=128, verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks)

Test

In [ ]:
# Predictions from the Test Set from the Trained Model
Predictions = Classification_Model.predict(X_Test, verbose=1)
print(Predictions.shape)

Error Performance

In [ ]:
# Error of the prediction, one of many evaluation metrics
# Using Mean Absolute Error (MAE) in this case as a sample
Error = mean_absolute_error(Y_Test, Predictions)
print(f"MAE: {Error}")

Plot Training History [Metrics] - Requires to Train the Model

In [ ]:
def history_plot(history):
  # list all dictionaries in history
  print(history.history.keys())
  # summarize history for error
  plt.figure(figsize=(12,10))
  plt.subplot(2,1,1)
  plt.plot(history.history['mse'])
  plt.plot(history.history['val_mse'])
  plt.title('Model Error Performance')
  plt.ylabel('Error')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()
  # summarize history for loss
  plt.figure(figsize=(12,10))
  plt.subplot(2,1,2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()
#
history_plot(history)

Convert raw predictions into single-column, integer-based predictions

In [ ]:
prediction_shape = Predictions.shape
prediction_length = prediction_shape[0]
Y_Preds = np.zeros((prediction_length, 1), dtype=int)
#
for i in range(0, prediction_length):
  prediction = Predictions[i]
  x = np.where(prediction == np.max(prediction))
  x = int(x[0])
  Y_Preds[i] = x

Y_Preds.shape

Print Confusion Matrix

In [ ]:
print('-------------------------------------------')
print('Raw Confusion Matrix')
print(confusion_matrix(Y_Test_, Y_Preds, normalize=None))
print('-------------------------------------------')
print('Normalized Confusion Matrix')
print(confusion_matrix(Y_Test_, Y_Preds, normalize='true'))
print('-------------------------------------------')

Plot Confusion Matrix

In [ ]:
# Use the numeric labels or replace with real-class names. But be careful about their respective sequence
labels = ['Class_0', 'Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7']

In [ ]:
def plot_conf_mat(Ground_Truth_Labels, Predictions):
  confusion_matrix_raw = confusion_matrix(Ground_Truth_Labels, Predictions, normalize=None)
  confusion_matrix_norm = confusion_matrix(Ground_Truth_Labels, Predictions, normalize='true')
  shape = confusion_matrix_raw.shape
  data = np.asarray(confusion_matrix_raw, dtype=int)
  text = np.asarray(confusion_matrix_norm, dtype=float)
  annots = (np.asarray(["{0:.2f} ({1:.0f})".format(text, data) for text, data in zip(text.flatten(), data.flatten())])).reshape(shape[0],shape[1])
  fig = plt.figure(figsize=(20, 10))
  sns.heatmap(confusion_matrix_norm, cmap='YlGnBu', annot=annots, fmt='', xticklabels=labels, yticklabels=labels)
  plt.title('Confusion Matrix', fontsize=25)
  plt.xlabel("Predicted", fontsize=15)
  plt.ylabel("Actual", fontsize=15)
#
plot_conf_mat(Y_Test_, Y_Preds)

Here it can be seen that risk level 6 and 7 could not be predicted well due to less number of instances.

In [ ]:
# Evaluating Overall Metrics - Accuracy, Precision, Recall, f1-Score
Accuracy = accuracy_score(Y_Test_, Y_Preds)
Precision = precision_score(Y_Test_, Y_Preds, average= 'weighted')
Recall = recall_score(Y_Test_, Y_Preds, average= 'weighted')
f1_Score = f1_score(Y_Test_, Y_Preds, average= 'weighted')
print(f'Accuracy = {Accuracy:.3f}')
print(f'Precision = {Precision:.3f}')
print(f'Recall = {Recall:.3f}')
print(f'f1-Score = {f1_Score:.3f}')

In [ ]:
print(classification_report(Y_Test_, Y_Preds, target_names=labels, zero_division=0))

Multiclass Receiver Operating Characteristic(ROC) Curves Plot

In [ ]:
def plot_multiclass_roc(Y_Test, Predictions):
  # Compute ROC curve and Area Under Curve (AUC) for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(class_number):
      fpr[i], tpr[i], _ = roc_curve(Y_Test[:, i], Predictions[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])

  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = roc_curve(Y_Test.ravel(), Predictions.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

  # First aggregate all false positive rates
  all_fpr = np.unique(np.concatenate([fpr[i] for i in range(class_number)]))
  # Then interpolate all ROC curves at this points
  mean_tpr = np.zeros_like(all_fpr)
  for i in range(class_number):
      mean_tpr += interp(all_fpr, fpr[i], tpr[i])

  # Finally average it and compute AUC
  mean_tpr /= class_number

  fpr["macro"] = all_fpr
  tpr["macro"] = mean_tpr
  roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

  # Plot all ROC curves
  plt.figure(figsize=(20, 10))
  plt.plot(fpr["micro"], tpr["micro"],
          label='micro-average ROC curve (area = {0:0.2f})'
                ''.format(roc_auc["micro"]),
          color='deeppink', linestyle=':', linewidth=4)

  plt.plot(fpr["macro"], tpr["macro"],
          label='macro-average ROC curve (area = {0:0.2f})'''.format(roc_auc["macro"]),
          color='navy', linestyle=':', linewidth=4)

  for i in range(class_number):
      plt.plot(fpr[i], tpr[i], lw=2,
              label='ROC curve of class {0} (area = {1:0.2f})'''.format(i, roc_auc[i]))

  plt.plot([0, 1], [0, 1], 'k--', lw=2)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate', fontsize=15)
  plt.ylabel('True Positive Rate', fontsize=15)
  plt.title('MultiClass ROC Plot with Respective AUC', fontsize=25)
  plt.legend(loc="lower right")
  plt.show()
#
plot_multiclass_roc(Y_Test, Predictions)

In [ ]:
def plot_multiclass_precision_recall_curves(Y_Test, Predictions):
  # For each class
  precision = dict()
  recall = dict()
  average_precision = dict()
  for i in range(class_number):
      precision[i], recall[i], _ = precision_recall_curve(Y_Test[:, i], Predictions[:, i])
      average_precision[i] = average_precision_score(Y_Test[:, i], Predictions[:, i])

  # A "micro-average": quantifying score on all classes jointly
  precision["micro"], recall["micro"], _ = precision_recall_curve(Y_Test.ravel(), Predictions.ravel())
  average_precision["micro"] = average_precision_score(Y_Test, Predictions, average="micro")
  print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision["micro"]))

  from itertools import cycle
  # setup plot details
  colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

  plt.figure(figsize=(20, 10))
  f_scores = np.linspace(0.2, 0.8, num=4)
  lines = []
  labels = []
  for f_score in f_scores:
      x = np.linspace(0.01, 1)
      y = f_score * x / (2 * x - f_score)
      l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
      plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

  lines.append(l)
  labels.append('iso-f1 curves')
  l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
  lines.append(l)
  labels.append('micro-average Precision-recall (area = {0:0.2f})'''.format(average_precision["micro"]))

  for i, color in zip(range(class_number), colors):
      l, = plt.plot(recall[i], precision[i], color=color, lw=2)
      lines.append(l)
      labels.append('Precision-recall for class {0} (area = {1:0.2f})'''.format(i, average_precision[i]))

  fig = plt.gcf()
  fig.subplots_adjust(bottom=0.25)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('Recall', fontsize=15)
  plt.ylabel('Precision', fontsize=15)
  plt.title('MultiClass Precision-Recall Curves', fontsize=25)
  plt.legend(lines, labels, loc=(0, -.3), prop=dict(size=14))
  plt.show()
#
plot_multiclass_precision_recall_curves(Y_Test, Predictions)